# Gene Expression Simple Demo

This shows how to query BgeeDb gene expression data ingested in Monarch

In [1]:
## Create an ontology factory in order to fetch Uberon
from ontobio.ontol_factory import OntologyFactory

ofactory = OntologyFactory()
ont = ofactory.create("uberon") 

In [6]:
## Create a sub-ontology that excludes all relations other than is-a and part-of
subont = ont.subontology(relations=['subClassOf', 'BFO:0000050'])

In [7]:
## Create an association factory to get mouse gene-expression associations (sourced from bgeedb)
from ontobio.assoc_factory import AssociationSetFactory
afactory = AssociationSetFactory()
aset = afactory.create(ontology=subont, subject_category='gene', object_category='anatomy', taxon='NCBITaxon:10090')


In [78]:
# show first 5
["{} '{}'".format(g, aset.label(g)) for g in aset.subjects[:5]]

["MGI:3646667 'Gm5134'",
 "MGI:1913305 'Sf3b6'",
 "MGI:3649022 'Rpl21-ps13'",
 "MGI:1353592 'Hacd1'",
 "MGI:3650210 'Gm13165'"]

In [11]:
# fetch uberon term
[liver] = ont.search('liver')
liver

'UBERON:0002107'

In [13]:
liver_genes = aset.query([liver])
["{} '{}'".format(g, aset.label(g)) for g in liver_genes]

["MGI:95590 'Ftl2-ps'",
 "MGI:1098641 'Wasf2'",
 "MGI:3644452 'Gm9083'",
 "MGI:3651858 'Gm11337'",
 "MGI:3702318 'Gm11989'",
 "MGI:1916396 'Gsdmd'",
 "MGI:2443767 'Aaas'",
 "MGI:2137698 'Ugt1a6a'",
 "MGI:1351659 'Abcg5'",
 "MGI:3705426 'Rpl21-ps15'",
 "MGI:1914745 'Tmem167b'",
 "MGI:1347249 'Psg16'",
 "MGI:3818630 'Sco2'",
 "MGI:1919235 'Acad10'",
 "MGI:3780170 'Gm2000'",
 "MGI:1343095 'Emc8'",
 "MGI:5454530 'Gm24753'",
 "MGI:3649027 'Gm7117'",
 "MGI:5454475 'Gm24698'",
 "MGI:4415000 'Gm16580'",
 "MGI:3646089 'Gm7803'",
 "MGI:97987 'Rnu3b3'",
 "MGI:1353433 'Timm8a1'",
 "MGI:3649865 'Rpl23a-ps14'",
 "MGI:106686 'Pon3'",
 "MGI:88216 'Btk'",
 "MGI:3704359 'Gm9803'",
 "MGI:3780980 'Gm2810'",
 "MGI:2140962 'Ugt2b34'",
 "MGI:2444981 'Phldb2'",
 "MGI:3644778 'Gm8738'",
 "MGI:88054 'Apoc2'",
 "MGI:3649201 'Gm12396'",
 "MGI:5451834 'Gm22057'",
 "MGI:1861354 'Apbb1ip'",
 "MGI:5804868 'Gm45753'",
 "MGI:891967 'Serpina1e'",
 "MGI:2138853 'AI182371'",
 "MGI:3041196 'Fam198a'",
 "MGI:1913761 'Chtop'

In [36]:
## NOTE: we currently lack rank scores, see https://github.com/monarch-initiative/monarch-app/issues/1271
## For now let's do something naive

def specificity_score(g, t):
    """
    Naive specificity score - penalize for every expression *not* in desired term, e.g. liver
    """
    anns = aset.annotations(g)
    nonspecific = [a for a in anns if t!=a and t not in subont.ancestors(a) and a not in subont.ancestors(g)]
    return 1/(len(nonspecific)+1)

## Tuples of (gene_id, gene_symbol, score)
gscores = [(g,aset.label(g),specificity_score(g,liver)) for g in liver_genes]
gscores
    

[('MGI:95590', 'Ftl2-ps', 0.2),
 ('MGI:1098641', 'Wasf2', 0.25),
 ('MGI:3644452', 'Gm9083', 0.125),
 ('MGI:3651858', 'Gm11337', 1.0),
 ('MGI:3702318', 'Gm11989', 0.25),
 ('MGI:1916396', 'Gsdmd', 0.25),
 ('MGI:2443767', 'Aaas', 0.25),
 ('MGI:2137698', 'Ugt1a6a', 0.125),
 ('MGI:1351659', 'Abcg5', 0.2),
 ('MGI:3705426', 'Rpl21-ps15', 0.25),
 ('MGI:1914745', 'Tmem167b', 0.16666666666666666),
 ('MGI:1347249', 'Psg16', 0.25),
 ('MGI:3818630', 'Sco2', 0.2),
 ('MGI:1919235', 'Acad10', 0.09090909090909091),
 ('MGI:3780170', 'Gm2000', 0.3333333333333333),
 ('MGI:1343095', 'Emc8', 0.25),
 ('MGI:5454530', 'Gm24753', 0.5),
 ('MGI:3649027', 'Gm7117', 0.2),
 ('MGI:5454475', 'Gm24698', 0.3333333333333333),
 ('MGI:4415000', 'Gm16580', 0.16666666666666666),
 ('MGI:3646089', 'Gm7803', 0.14285714285714285),
 ('MGI:97987', 'Rnu3b3', 0.25),
 ('MGI:1353433', 'Timm8a1', 0.25),
 ('MGI:3649865', 'Rpl23a-ps14', 0.16666666666666666),
 ('MGI:106686', 'Pon3', 0.5),
 ('MGI:88216', 'Btk', 0.3333333333333333),
 ('MGI:

In [38]:
sorted(gscores, key=lambda x: -x[2])

[('MGI:3651858', 'Gm11337', 1.0),
 ('MGI:5804868', 'Gm45753', 1.0),
 ('MGI:5453898', 'Gm24121', 1.0),
 ('MGI:3779824', 'Gm8941', 1.0),
 ('MGI:1196423', 'Onecut1', 1.0),
 ('MGI:5453406', 'Gm23629', 1.0),
 ('MGI:4834232', 'Mir3060', 1.0),
 ('MGI:3648378', 'Sult2a5', 1.0),
 ('MGI:3646055', 'Gm5801', 1.0),
 ('MGI:3579898', 'Rhox8', 1.0),
 ('MGI:5452188', 'Gm22411', 1.0),
 ('MGI:5455261', 'Gm25484', 1.0),
 ('MGI:3651700', 'Gm11752', 1.0),
 ('MGI:103164', 'Mif-ps3', 1.0),
 ('MGI:4937289', 'Gm17655', 1.0),
 ('MGI:3780193', 'Mup12', 1.0),
 ('MGI:5455227', 'Gm25450', 1.0),
 ('MGI:88603', 'Cyp2d11', 1.0),
 ('MGI:1920648', 'Nipsnap3a', 1.0),
 ('MGI:5530773', 'Mir6353', 1.0),
 ('MGI:3652220', 'Hnf1aos2', 1.0),
 ('MGI:5455658', 'Gm25881', 1.0),
 ('MGI:4834247', 'Mir3074-1', 1.0),
 ('MGI:3651800', 'Gm14373', 1.0),
 ('MGI:5521072', 'Zfp-ps', 1.0),
 ('MGI:5453401', 'Gm23624', 1.0),
 ('MGI:2387153', 'Npb', 1.0),
 ('MGI:5452105', 'Gm22328', 1.0),
 ('MGI:3783155', 'Gm15713', 1.0),
 ('MGI:5455351', 'Gm255

In [40]:
only_in_liver = [x for x in gscores if x[2] == 1.0]
only_in_liver

[('MGI:3651858', 'Gm11337', 1.0),
 ('MGI:5804868', 'Gm45753', 1.0),
 ('MGI:5453898', 'Gm24121', 1.0),
 ('MGI:3779824', 'Gm8941', 1.0),
 ('MGI:1196423', 'Onecut1', 1.0),
 ('MGI:5453406', 'Gm23629', 1.0),
 ('MGI:4834232', 'Mir3060', 1.0),
 ('MGI:3648378', 'Sult2a5', 1.0),
 ('MGI:3646055', 'Gm5801', 1.0),
 ('MGI:3579898', 'Rhox8', 1.0),
 ('MGI:5452188', 'Gm22411', 1.0),
 ('MGI:5455261', 'Gm25484', 1.0),
 ('MGI:3651700', 'Gm11752', 1.0),
 ('MGI:103164', 'Mif-ps3', 1.0),
 ('MGI:4937289', 'Gm17655', 1.0),
 ('MGI:3780193', 'Mup12', 1.0),
 ('MGI:5455227', 'Gm25450', 1.0),
 ('MGI:88603', 'Cyp2d11', 1.0),
 ('MGI:1920648', 'Nipsnap3a', 1.0),
 ('MGI:5530773', 'Mir6353', 1.0),
 ('MGI:3652220', 'Hnf1aos2', 1.0),
 ('MGI:5455658', 'Gm25881', 1.0),
 ('MGI:4834247', 'Mir3074-1', 1.0),
 ('MGI:3651800', 'Gm14373', 1.0),
 ('MGI:5521072', 'Zfp-ps', 1.0),
 ('MGI:5453401', 'Gm23624', 1.0),
 ('MGI:2387153', 'Npb', 1.0),
 ('MGI:5452105', 'Gm22328', 1.0),
 ('MGI:3783155', 'Gm15713', 1.0),
 ('MGI:5455351', 'Gm255

In [51]:
## get phenotype associations
mp = ofactory.create("mp")

pheno_aset = afactory.create(ontology=mp, subject_category='gene', object_category='phenotype', taxon='NCBITaxon:10090')


In [77]:
## Show phenotype anns for all genes in liver

for g in liver_genes:
    anns = pheno_aset.annotations(g)
    print("{} {} {}".format(g,aset.label(g), [(a,mp.label(a)) for a in anns]))
    

MGI:95590 Ftl2-ps []
MGI:1098641 Wasf2 [('MP:0002188', 'small heart'), ('MP:0020329', 'decreased capillary density'), ('MP:0011091', 'prenatal lethality, complete penetrance'), ('HP:0002170', None), ('HP:0000969', None), ('MP:0003984', 'embryonic growth retardation'), ('MP:0000260', 'abnormal angiogenesis'), ('MP:0011098', 'embryonic lethality during organogenesis, complete penetrance'), ('MP:0001722', 'pale yolk sac'), ('MP:0000295', 'trabecula carnea hypoplasia'), ('MP:0008803', 'abnormal placental labyrinth vasculature morphology'), ('GO:0001667PHENOTYPE', None), ('HP:0025016', None), ('MP:0000822', 'abnormal brain ventricle morphology'), ('GO:0006928PHENOTYPE', None), ('MP:0003974', 'abnormal endocardium morphology'), ('GO:0001525PHENOTYPE', None), ('MP:0004251', 'failure of heart looping')]
MGI:3644452 Gm9083 []
MGI:3651858 Gm11337 []
MGI:3702318 Gm11989 []
MGI:1916396 Gsdmd [('MP:0011073', 'abnormal macrophage apoptosis')]
MGI:2443767 Aaas [('MP:0005379', 'endocrine/exocrine glan

## More advanced expression analyses

For more advanced analyses, see http://bgee.unil.ch